In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

In [ ]:

df = pd.read_csv('/kaggle/input/sleep-apnea/a01.csv')
df.columns =['Time', 'ECG']
# print(df) #[2956999 rows x 2 columns]


df1 = pd.read_csv('/kaggle/input/sleep-apnea/a01.txt', delimiter= '\s+', index_col=False)
df1Arr = df1.loc[:,'N']
# print(df) #[2956999 rows x 2 columns]

In [ ]:
#module for data signals synchronization with labels 
yArr = []
i = 0
for x in df1Arr:
    for k in range(0,6060):
        yArr.append(x)
        i+=1

df_y = pd.DataFrame(yArr)
df = pd.concat([df, df_y], axis=1)

df = df[:2956999]
df.rename(columns = {0 : 'Target'}, inplace = True)
df["Target"].replace({"A": 1, "N": 0}, inplace=True)

y = df.values[:2956999, 2]


df = df.values[:2956999, 1]
x = df

k = 6000
step = 3000
n = len(x)

# max, min, median, mean, std, kurtosis, skw, target
dfk = pd.DataFrame(columns=['max', 'min', 'median', 'mean', 'std', 'kts', 'skw', 'target'])
kk = 0
# count = 0

window = x[0:0 + k]
peaks, _ = find_peaks(window, distance=150) #to finds r_peaks and finds all local maxima by
# simple comparison of neighbouring values


for i in range(0, n - k + 1, step): #to iterate within all window segments
     # count =count+1
     window = x[i:i + k]
     windowY = y[i:i + k]
     peaks, _ = find_peaks(window, distance=150) #to finds r_peaks and finds all local maxima by
    # simple comparison of neighbouring values
     ecg_peaks = window[peaks] #to get amplitude information
     max_index = np.where(ecg_peaks == np.amax(ecg_peaks))
     windowPeakIdx = peaks[max_index[0]]
         
    # print(x[25])
    # plt.plot(x)
    # plt.plot(peaks, x[peaks], ".")
    # plt.plot(np.zeros_like(x), "--", color="gray")
    # plt.show()
    
     if ecg_peaks.size == 0:
      kk += 1
      continue
      
    # The electrocardiogram in millivolt (mV) sampled at 100 Hz.
    
     fs = 100
    # np.diff(peaks)
     ecg_sample_time = peaks / fs #sampling data converted into time domain data
     # print(ecg_sample_time) 
     ecg_sample_time_interval = [ecg_sample_time[i + 1] - ecg_sample_time[i] for i in range(len(ecg_sample_time) - 1)] #time interval between a signal and its previous signal
     # print(ecg_sample_time_interval) 
     
    #  feature extraction for every window segment
     G = max(window[peaks])
     dfk.at[kk, 'max'] = G #maximum of r_peaks
     # print(G)
     L = min(window[peaks]) #minimum of r_peaks
     dfk.at[kk, 'min'] = L
     # print(L)
     mean= np.mean(ecg_sample_time_interval) #mean of r_peaks
     dfk.at[kk,'mean'] = mean
     # print(mean)
     median= np.median(ecg_sample_time_interval) #median of r_peaks
     dfk.at[kk,'median'] = median
     # print(median)
     std= np.std(ecg_sample_time_interval) #std deviation of r_peaks
     dfk.at[kk,'std'] = std
     # print(std)
     kts= kurtosis(ecg_sample_time_interval, fisher=True) #kurtosis of r_peaks
     dfk.at[kk,'kts'] = kts
     # print(kts)
     skw= skew(ecg_sample_time_interval) #skwewness of r_peaks
     dfk.at[kk,'skw'] = skw
     # print(skw)
     
     dfk.at[kk,'target'] = windowY[windowPeakIdx][0]
     kk+=1
# print(dfk)

In [ ]:
dft = np.transpose(dfk)
# print(dft.shape)
# plt.boxplot(dft)
# plt.xlabel('feature')
# plt.ylabel('value')
# plt.show()

x1 = dft.iloc[0:6,:]

y1 = dft.iloc[7,:]
y1= y1.astype(int)
# print(y1.value_counts())

x1 = np.transpose(x1)

In [ ]:
from sklearn import preprocessing

# normalize the data attributes
x1 = preprocessing.normalize(x1)

x_train_all, x_test, y_train_all, y_test = train_test_split(x1, y1,  
                                                            test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify=y_train_all, 
                                                  test_size=0.2, random_state=42)
# print(y_train.value_counts())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
scaler.fit(x_val)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
model = Sequential()

model.add(Dense(40,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(20,activation='relu'))
model.add(Dropout(0.2))

# BINARY CLASSIFICATION so use sigmoid for the last layer
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=25)

In [ ]:
model.fit(x=x_train,y=y_train,epochs=200,validation_data=(x_val,y_val),batch_size=30,callbacks=[early_stop])

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
predictions = model.predict_classes(x_val)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_val,predictions))
print(confusion_matrix(y_val,predictions))

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
neural_net_model = model.save('SleepApnea-predictor.h5') # save model
model_loss